## **XGBoost Regression**
https://rdrr.io/cran/xgboost/man/xgb.cv.html

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 100000

import gc
from myUtils import *
from feature_generator import feature_v1, feature_v2, feature_v3
import xgboost as xgb

In [20]:
###################################################################
# make features
##################################################################
feature_maker = feature_v3.FeaturesMaker_v3(target_col="item_cnt")

base_data = "sales_train_eval"
data = pd.read_pickle(os.path.join("mydata",base_data+".pickle"))
#data = pd.read_pickle(os.path.join("mydata","sales_train_eval.pickle"))

data = feature_maker.make_feature(data)

data_path = os.path.join("mydata",base_data+"_"+feature_maker.name+".pickle")
with open(data_path,"wb") as f:
    pickle.dump(data,f)

-- column check completed --
  columns are satisfied
--  features_ver3  --
dim: 25
N: 60034810
-----------------


In [22]:
print("evaluation")
print(data["evaluation"][0].isna().sum())
print()

print("validation")
print(data["validation"][0].isna().sum())
print()

print("train")
print(data["train"][0].isna().sum())
print()

evaluation
item_id                          0
dept_id                          0
cat_id                           0
store_id                         0
state_id                         0
sold_lag_28                      0
sold_lag_35                      0
sold_lag_42                      0
sold_lag_49                      0
sold_lag_56                      0
item_sold_avg                    0
state_sold_avg                   0
store_sold_avg                   0
cat_sold_avg                     0
dept_sold_avg                    0
cat_dept_sold_avg                0
store_item_sold_avg              0
cat_item_sold_avg                0
dept_item_sold_avg               0
state_store_sold_avg             0
state_store_cat_sold_avg         0
store_cat_dept_sold_avg          0
rolling_sold_mean           853720
expanding_sold_mean              0
selling_trend               853720
dtype: int64

validation
item_id                          0
dept_id                          0
cat_id             

In [24]:
model_path = os.path.join("models","XGBoost_"+feature_maker.name+".mdl")

if os.path.exists(model_path):
    print("loading trained model...")
    model = xgb.XGBRegressor()
    model.load_model(model_path)
    
else:
    print("start training XGBoost")
    model = xgb.XGBRegressor(objective="reg:squarederror",
                             tree_method="gpu_hist",
                             random_state=0,
                             verbose=2,
                             n_estimators=1000,
                             )

    model.fit(X=data["train"][0], y=data["train"][1], 
              #sample_weight=None, 
              #base_margin=None, 
              eval_set=[data["train"],data["validation"]], 
              #eval_metric=None, 
              early_stopping_rounds=100, 
              verbose=True, 
              #xgb_model=None, 
              #sample_weight_eval_set=None
              )
    model.save_model(model_path)
print("  -- completed\n")

start training XGBoost


OSError: [WinError -529697949] Windows Error 0xe06d7363

#### [prediction]

In [6]:
# prediction
print("start prediction")
pred_mask = data["evaluation"][1].isna()
data["evaluation"][1].loc[pred_mask] = model.predict(data["evaluation"][0])
print("  -- completed\n")

start prediction
[22:22:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:979: Number of columns does not match number of features in booster. Columns: 25 Features: 27
  -- completed



In [17]:
data["evaluation"][0]

,,item_id,dept_id,cat_id,store_id,state_id,sold_lag_28,sold_lag_35,sold_lag_42,sold_lag_49,sold_lag_56,...,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg,rolling_sold_mean,expanding_sold_mean,selling_trend
id,d,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,d_1942,1437,3,1,0,0,NaN,1.0,0.0,0.0,1.0,...,0.703125,0.313721,0.213989,0.213989,1.320312,0.811035,1.032227,NaN,0.313721,NaN
HOBBIES_1_002_CA_1_evaluation,d_1942,1438,3,1,0,0,NaN,0.0,0.0,0.0,1.0,...,0.703125,0.257812,0.264404,0.264404,1.320312,0.811035,1.032227,NaN,0.257812,NaN
HOBBIES_1_003_CA_1_evaluation,d_1942,1439,3,1,0,0,NaN,1.0,0.0,0.0,0.0,...,0.703125,0.150513,0.075012,0.075012,1.320312,0.811035,1.032227,NaN,0.150513,NaN
HOBBIES_1_004_CA_1_evaluation,d_1942,1440,3,1,0,0,NaN,4.0,2.0,1.0,0.0,...,0.703125,1.718750,2.046875,2.046875,1.320312,0.811035,1.032227,NaN,1.718750,NaN
HOBBIES_1_005_CA_1_evaluation,d_1942,1441,3,1,0,0,NaN,0.0,2.0,0.0,1.0,...,0.703125,0.966309,0.764160,0.764160,1.320312,0.811035,1.032227,NaN,0.966309,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,d_1969,1432,2,0,9,2,NaN,NaN,NaN,NaN,1.0,...,2.056641,0.534180,0.804199,0.804199,1.101562,1.720703,2.236328,NaN,0.534180,NaN
FOODS_3_824_WI_3_evaluation,d_1969,1433,2,0,9,2,NaN,NaN,NaN,NaN,0.0,...,2.056641,0.376465,0.435059,0.435059,1.101562,1.720703,2.236328,NaN,0.376465,NaN
FOODS_3_825_WI_3_evaluation,d_1969,1434,2,0,9,2,NaN,NaN,NaN,NaN,0.0,...,2.056641,0.895020,0.707031,0.707031,1.101562,1.720703,2.236328,NaN,0.895020,NaN


In [16]:
tmp = data["evaluation"][1].reset_index()
mask = tmp["id"] == "HOBBIES_1_001_CA_1_evaluation"

tmp[mask]

,id,d,item_cnt
0,HOBBIES_1_001_CA_1_evaluation,d_1942,0.364502
30490,HOBBIES_1_001_CA_1_evaluation,d_1943,0.364502
60980,HOBBIES_1_001_CA_1_evaluation,d_1944,0.364502
91470,HOBBIES_1_001_CA_1_evaluation,d_1945,0.364502
121960,HOBBIES_1_001_CA_1_evaluation,d_1946,0.364502
152450,HOBBIES_1_001_CA_1_evaluation,d_1947,0.364502
182940,HOBBIES_1_001_CA_1_evaluation,d_1948,0.364502
213430,HOBBIES_1_001_CA_1_evaluation,d_1949,0.364502
243920,HOBBIES_1_001_CA_1_evaluation,d_1950,0.364502
274410,HOBBIES_1_001_CA_1_evaluation,d_1951,0.364502


#### [submission]

In [71]:
# submission 
print("start submission")
sub_path = os.path.join("submission_point","XGBoost_"+feature_maker.name+"_submission.csv")


sub_cols = ["id"] + [f"F{i}" for i in range(1, 29)]

valid = data["validation"][1]
evalu = data["evaluation"][1]

del data
gc.collect()

valid = pd.DataFrame(valid.values,
                     index=valid.index,
                     columns=[feature_maker.target_col])
evalu = pd.DataFrame(evalu.values,
                     index=evalu.index,
                     columns=[feature_maker.target_col])

valid = valid.reset_index()
evalu = evalu.reset_index()

valid = pd.pivot(valid,
                 index="id", 
                 columns="d",
                 values=feature_maker.target_col)
evalu = pd.pivot(evalu,
                 index="id", 
                 columns="d", 
                 values=feature_maker.target_col)

valid = valid.reset_index()
evalu = evalu.reset_index()

valid.columns = sub_cols
evalu.columns = sub_cols

valid["id"] = valid["id"].str.replace("_evaluation","_validation")

pd.concat([valid,evalu]).to_csv(sub_path,index=False)
print("  -- completed")

start submission
  -- completed
